In [ ]:
import signal
import requests
from time import sleep
import numpy as np
import sys

class ApiException(Exception):
    pass

def signal_handler(sigum, frame):
    global shutdown 
    signal.signal(signal.SIGINT, signal.SIG_DFL)
    shutdown=True
    
API_KEY = {'X-API-Key': 'YA6YJ9TB'}
shutdown=False

s = requests.session() 
s.headers.update(API_KEY)

#Global Variables
SPEEDBUMP = 0.5
MAX_LONG_EXPOSURE_NET = 25000
MAX_SHORT_EXPOSURE_NET = -25000
MAX_EXPOSURE_GROSS = 500000
ORDER_LIMIT = 5000
MAX_ORDERS = 5
ticker_list = ['UB','GEM','ETF']

def get_tick(s):   
    resp = s.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick'], case['status']
    raise ApiException('Authorization error, check key')


def get_bid_ask(ticker):
    payload = {'ticker': ticker}
    resp = s.get('http://localhost:9999/v1/securities/book', params = payload)
    if resp.ok:
        book = resp.json()
        bid_side_book = book['bids']
        ask_side_book = book['asks']
        
        bid_prices_book = [item["price"] for item in bid_side_book]
        ask_prices_book = [item['price'] for item in ask_side_book]
        
        best_bid_price = bid_prices_book[0]
        best_ask_price = ask_prices_book[0]
  
        return best_bid_price, best_ask_price

def get_time_sales(ticker):
    payload = {'ticker': ticker}
    resp = s.get('http://localhost:9999/v1/securities/tas', params = payload)
    if resp.ok:
        book = resp.json()
        time_sales_book = [item["quantity"] for item in book]
        return time_sales_book

def get_position():
    resp = s.get('http://localhost:9999/v1/securities')
    if resp.ok:
        book = resp.json()
        gross_position = abs(book[0]['position']) + abs(book[1]['position']) + abs(book[2]['position'])
        net_position = book[0]['position'] + book[1]['position'] + book[2]['position']
        return gross_position, net_position

def profit(ticker):
    resp = s.get('http://localhost:9999/v1/securities')
    if resp.ok:
        book = resp.json()
        profit = book[ticker]['Unrealized PL']
        return profit
    
def get_open_orders(ticker):
    payload = {'ticker': ticker}
    resp = s.get('http://localhost:9999/v1/orders', params = payload)
    if resp.ok:
        orders = resp.json()
        buy_orders = [item for item in orders if item["action"] == "BUY"]
        sell_orders = [item for item in orders if item["action"] == "SELL"]
        return buy_orders, sell_orders
    
def get_order_status(order_id):
    resp = s.get('http://localhost:9999/v1/orders' + '/' + str(order_id))
    if resp.ok:
        order = resp.json()
        return order['status']

def get_news(estimates_data,news_query_length): 
    resp = s.get ('http://localhost:9999/v1/news')
    if resp.ok:
        news_query = resp.json()
        news_query_length_check = len(news_query)
        
        if news_query_length_check > news_query_length:
            news_query_length = news_query_length_check
            
            newest_tick = news_query[0]['tick']
            start_char = news_query[0]['body'].find("$")
            newest_estimate = float(news_query[0]['body'][start_char + 1 : start_char + 6])
                        
            if news_query[0]['headline'].find("UB") > 0:
                estimates_data[0,0] = max(newest_estimate - ((300 - newest_tick) / 50), estimates_data[0,0])
                estimates_data[0,1] = min(newest_estimate + ((300 - newest_tick) / 50), estimates_data[0,1])
           
            elif news_query[0]['headline'].find("GEM") > 0:
                estimates_data[1,0] = max(newest_estimate - ((300 - newest_tick) / 50), estimates_data[1,0])
                estimates_data[1,1] = min(newest_estimate + ((300 - newest_tick) / 50), estimates_data[1,1])
            
        estimates_data[2,0] = estimates_data[0,0] + estimates_data[1,0]
        estimates_data[2,1] = estimates_data[0,1] + estimates_data[1,1]
                
        return estimates_data, news_query_length
    
def main():
    #creates a session to manage connections and requests to the ril client
    tick, status = get_tick(s)
    ticker_list = ['UB','GEM','ETF']
    market_prices = np.array([0.,0.,0.,0.,0.,0.])
    market_prices = market_prices.reshape(3,2)
    
    news_query_length = 1
    estimates_data = np.array([40., 60., 20., 30., 60., 90.])
    estimates_data = estimates_data.reshape(3,2)
 
    while (tick > 5 and tick < 295 and not shutdown) and status == 'ACTIVE':
        for i in range(3):
            
            ticker_symbol = ticker_list[i]
            market_prices[i,0], market_prices[i,1] = get_bid_ask(ticker_symbol)
        
            estimates_data, news_query_length = get_news(estimates_data, news_query_length)
            gross_position, net_position = get_position()
            if market_prices[0, 0] + market_prices[1, 0] > market_prices[2, 1] + 0.05:
                print('c')
            if market_prices[0, 1] + market_prices[1, 1] < market_prices[2, 0] + 0.05:
                print('r')
        
            print(estimates_data)
        
            if gross_position < MAX_EXPOSURE_GROSS:
            
                if market_prices[0, 0] > (estimates_data[0, 1] - 0.25): 
                    resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'UB', 'type': 'LIMIT', 'quantity': ORDER_LIMIT, 'price': market_prices[0, 0], 'action': 'SELL'})
                    if (book[0]['Unrealized']) >= 30000:
                        resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'UB', 'type': 'LIMIT', 'quantity': ORDER_LIMIT, 'price': market_prices[0, 1], 'action': 'BUY'})
                
                if market_prices[0, 1] < (estimates_data[0, 0] + 0.25): 
                    resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'UB', 'type': 'LIMIT', 'quantity': ORDER_LIMIT, 'price': market_prices[0, 1], 'action': 'BUY'})                
                    if (book[0]['Unrealized']) >= 30000:
                        resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'UB', 'type': 'LIMIT', 'quantity': ORDER_LIMIT, 'price': market_prices[0, 0], 'action': 'SELL'})
                
                if market_prices[1, 0] > (estimates_data[1, 1] - 0.25): 
                    resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'GEM', 'type': 'LIMIT', 'quantity': ORDER_LIMIT, 'price': market_prices[1, 0], 'action': 'SELL'})
                    if (book[1]['Unrealized']) >= 30000:
                        resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'GEM', 'type': 'LIMIT', 'quantity': ORDER_LIMIT, 'price': market_prices[1, 1], 'action': 'BUY'})                
                    
                if market_prices[1, 1] < (estimates_data[1, 0] + 0.25): 
                    resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'GEM', 'type': 'LIMIT', 'quantity': ORDER_LIMIT, 'price': market_prices[1, 1], 'action': 'BUY'})                
                    if (book[1]['Unrealized']) >= 30000:
                        resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'GEM', 'type': 'LIMIT', 'quantity': ORDER_LIMIT, 'price': market_prices[1, 0], 'action': 'SELL'})
                
                if market_prices[2, 0] > (estimates_data[2, 1] - 0.25): 
                    resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'ETF', 'type': 'LIMIT', 'quantity': ORDER_LIMIT, 'price': market_prices[2, 0], 'action': 'SELL'})
                    if (book[2]['Unrealized']) >= 30000:
                        resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'ETF', 'type': 'LIMIT', 'quantity': ORDER_LIMIT, 'price': market_prices[2, 1], 'action': 'BUY'})                  
                    
                if market_prices[2, 1] < (estimates_data[2, 0] + 0.25): 
                    resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'ETF', 'type': 'LIMIT', 'quantity': ORDER_LIMIT, 'price': market_prices[2, 1], 'action': 'BUY'})                  
                    if (book[2]['Unrealized']) >= 30000:
                        resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'ETF', 'type': 'LIMIT', 'quantity': ORDER_LIMIT, 'price': market_prices[2, 0], 'action': 'SELL'})
                    
                        
        sleep(0.5)

        for i in range(3):
            
            ticker_symbol = ticker_list[i]          
            s.post('http://localhost:9999/v1/commands/cancel', params = {'ticker': ticker_symbol})
        
        tick, status = get_tick(s)
     
if __name__ == '__main__':
    main()